In [42]:
import numpy as np
from subphot_credentials import *
import time,resource
from datetime import date,timedelta
import requests
import re
import json
import urllib
import pandas as pd

In [43]:

# smtp2go_api_key = 'api-399081350B924C738533D3BCEB5096A1'
token="549e219d-b0eb-4a19-98ee-3a658af2303b"


def api(method,endpoint,data=None):
    headers={"Authorization":f"token {token}"}
    response=requests.request(method.upper(),urllib.parse.urljoin("https://fritz.science", endpoint),json=data,headers=headers)

    return response

def SN_data_phot(name):
    '''Retrieves individual events, form needs to be an array or a list, puts data into a dataframe'''
    data_array = []

    # if 'ZTF' in name:
    while True:
        response = api("get", f"api/sources/{name}/photometry")
        if response.status_code==200:
            break
    data = response.json().get("data", None)
    if len(data) ==0:
        df = pd.DataFrame()
    else:
        df = pd.DataFrame(data).sort_values(by=['mjd'], ascending=True)

    return df
    
SN_data_phot('2025bbo')

,obj_id,ra,dec,filter,mjd,snr,instrument_id,instrument_name,ra_unc,dec_unc,origin,id,altdata,created_at,groups,mag,magerr,magsys,limiting_mag
0,2025bbo,24.32197,45.7255,atlaso,60533.586157,NaN,72,ATLAS,None,None,fp,672193344,None,2025-02-09T01:26:13.907117,"[{'id': 41, 'name': 'Redshift Completeness Fac...",NaN,NaN,ab,19.4500
1,2025bbo,24.32197,45.7255,atlaso,60533.588188,NaN,72,ATLAS,None,None,fp,672193345,None,2025-02-09T01:26:13.907181,"[{'id': 41, 'name': 'Redshift Completeness Fac...",NaN,NaN,ab,19.1800
2,2025bbo,24.32197,45.7255,atlaso,60533.593508,NaN,72,ATLAS,None,None,fp,672193346,None,2025-02-09T01:26:13.907199,"[{'id': 41, 'name': 'Redshift Completeness Fac...",NaN,NaN,ab,19.4000
3,2025bbo,24.32197,45.7255,atlaso,60533.607458,NaN,72,ATLAS,None,None,fp,672193347,None,2025-02-09T01:26:13.907213,"[{'id': 41, 'name': 'Redshift Completeness Fac...",NaN,NaN,ab,19.4700
4,2025bbo,24.32197,45.7255,atlasc,60535.549141,NaN,72,ATLAS,None,None,fp,672193348,None,2025-02-09T01:26:13.907227,"[{'id': 41, 'name': 'Redshift Completeness Fac...",NaN,NaN,ab,19.4400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,2025bbo,NaN,NaN,ztfg,60716.226632,NaN,1,ZTF,None,None,fp,672460245,None,2025-02-11T00:43:58.302515,"[{'id': 1, 'name': 'Sitewide Group', 'nickname...",NaN,NaN,ab,20.0817
365,2025bbo,NaN,NaN,sdssr,60716.619097,10.857362,64,GITCamera,None,None,None,672460209,None,2025-02-10T22:06:52.012785,"[{'id': 1, 'name': 'Sitewide Group', 'nickname...",19.89,0.10,ab,20.1200
366,2025bbo,NaN,NaN,sdssg,60716.840900,1.085736,33,IOO,None,None,None,672464168,None,2025-02-11T03:13:52.996281,"[{'id': 1, 'name': 'Sitewide Group', 'nickname...",23.00,1.00,ab,21.4600
367,2025bbo,NaN,NaN,sdssr,60716.845140,9.047802,33,IOO,None,None,None,672464166,None,2025-02-11T03:13:52.996210,"[{'id': 1, 'name': 'Sitewide Group', 'nickname...",20.67,0.12,ab,21.8200


In [44]:
def delete_photometry(phot_id_data):
    response = api("delete", f"api/photometry/{phot_id_data['phot_id']}")
    if str(response.status_code)=='200':
        print(f" Successfully deleted photometry for phot ID: {phot_id_data['phot_id']}, mjd: {phot_id_data['mjd']}")
    else:
        print(f" Unable to delete photometry for phot ID: {phot_id_data['phot_id']},mjd: {phot_id_data['mjd']} STATUS CODE={response.status_code}")


def save_to_group(obj_id):
    save_group_id = '1754'
    save_url = "http://localhost:5000/api/source_groups"

    save_payload = {
        "objId": obj_id,
        "inviteGroupIds": [save_group_id],
        "unsaveGroupIds": []
    }

    save_headers = {"Content-Type": "application/json"}

    # save_response = requests.post(save_url, json=save_payload, headers=save_headers)
    save_response = requests.post(url=f"https://fritz.science/api/source_groups",headers = {'Authorization': f'token {token}'} ,json=save_payload)

    if save_response.status_code==200:print(f" Saved {obj_id} to group {save_group_id}")
    else:print(f" Unable to save {obj_id} to group {save_group_id}, STATUS CODE={save_response.status_code}")
    return


def upload_phot(args):
    sci_obj = args['sci_obj']
    SNR = args['SNR']
    sci_filt = args['sci_filt']
    mag_all_err = args['mag_all_err']
    lim_mag = args['lim_mag']
    mag = args['mag']
    sci_mjd = args['sci_mjd']
    sci_prop = args['sci_prop']
    telescope = args['telescope']
    if_stacked = args['if_stacked']
    sci_exp_time = args['sci_exp_time']
    sci_seeing = args['sci_seeing']


    print(f" Attempting to upload photometry")

    # print(mag+mag_all_err,lim_mag,mag+mag_all_err>lim_mag)
    if SNR<=3 or mag+mag_all_err>lim_mag:
        print(f' S/N of {SNR:.2f} or mag+magerr of {mag+mag_all_err:.2f} > maglim of {lim_mag:.2f}, uploading limit')
        # mag=[99.0,90.0,99.0,lim_mag]

    data = {"filter":f"sdss{sci_filt}","magerr": mag_all_err,"obj_id": sci_obj,
                "mag":mag,"limiting_mag": lim_mag,"mjd": sci_mjd,"magsys": "ab","group_ids":"all"}
    # return 

    if telescope in ['LT','lt','Liverpool','Liverpool Telescope','liverpool telescope','liverpool','liverpooltelescope']:data['instrument_id'],data['origin']='33','LT_IOO_PIPE'
    # elif telescope in SEDM:data['instruement_id'],data['origin']='2','SEDM_SUBPHOT_KPIPE'
    elif telescope in ['SLT','Lulin','slt','lulin']:data['instrument_id'],data['origin']='1104','SLT_SUBPHOT_KPIPE'
    elif telescope in ['TJO','tjo','MEIA3','meia3']:data['instrument_id'],data['origin']='1103','TJO_SUBPHOT_KPIPE'
    elif telescope in ['OSIRIS','osiris','GTC-OSIRIS','gtc-osiris']:data['instrument_id'],data['origin']='37','OSIRIS_SUBPHOT_KPIPE'
    elif telescope in ['HIPERCAME','hipercame','GTC-HIPERCAM','gtc-hipercam']:data['instrument_id'],data['origin']='1105','HIPERCAM_SUBPHOT_KPIPE'
    upload_new=True
    data['altdata'] = {}
    if if_stacked==True:data['altdata']['stacked'],data['altdata']['no_in_stack']= True,no_stacked

    data['altdata']['Reducer'] = 'K-Ryan Hinds'
    data['altdata']['Proposal PI'] = 'Dan Perley'
    data['altdata']['exptime'] = sci_exp_time
    data['altdata']['seeing'] = sci_seeing
    if sci_prop!=None:
        data['altdata']['Proposal ID']=sci_prop
        if sci_prop in ['JL24A04','JL24B14']:data['altdata']['Proposal PI'] = 'K-Ryan Hinds'
        elif sci_prop in ['JL24B15','JL25A01']:data['altdata']['Proposal PI'] = 'Jacob Wise'
        elif sci_prop in ['JL24B10']:data['altdata']['Proposal PI'] = 'Chris Copperwheat'


    name=sci_obj

    if any(X in name for X in ['-ugriz','-griz','-gri']):
        name = re.sub(r'-ugriz|-griz|-gri', '', name)

    if name.startswith('SN') or name.startswith('AT'):
        name = name[2:]

    data['obj_id'] = name

    [print(key+': '+str(item)) for key,item in data.items()]
    print(f' Fritz page: https://fritz.science/source/{name}')

    if mag>40:
        print(f' Magnitude of {mag} recorded, not uploading magnitude')
        if lim_mag<30:
            print(f' Limiting magnitude of {np.round(lim_mag,3)} recorded, attempting to upload, checking if already uploaded')
            all_fritz_data = SN_data_phot(name)
            fritz_ind = [ind for ind,val in enumerate(all_fritz_data['instrument_id']) if val==33 and all_fritz_data['filter'].iloc[ind]==data['filter']]

            on_fritz_mjd = [np.round(all_fritz_data['mjd'].iloc[ind],6) for ind in fritz_ind]
            on_fritz_id = [all_fritz_data['id'].iloc[ind] for ind in fritz_ind]
            on_fritz_mag = [np.round(all_fritz_data['limiting_mag'].iloc[ind],3) for ind in fritz_ind]
            upload_new=True  #identifier to update photometry in the case of stacking

            if sci_mjd in on_fritz_mjd:
                print(f' Limiting magnitude of {np.round(lim_mag,3)} already uploaded, checking if it is the same')
                upload_new=False
                if any(abs(on_fritz_mag[ind]-lim_mag)<0.1 for ind in range(len(on_fritz_mag))) or any(on_fritz_mag[ind]==np.round(lim_mag,3) for ind in range(len(on_fritz_mag))):
                    print(f' Limiting magnitude of {np.round(lim_mag,3)} already uploaded, not uploading')
                    sys_exit=True 
                    return
                else:   
                    print(f' Limiting magnitude of {np.round(lim_mag,3)} not uploaded, uploading')
                    upload_new=True

            if upload_new==True:
                data = {'filter':f"sdss{sci_filt}",'mag':None,'magerr':None,'mjd':sci_mjd,'limiting_mag_nsigma':5,'obj_id':sci_obj,'origin':'LT_IOO_PIPE',
                                    'magsys':'ab','group_ids':'all','limiting_mag':lim_mag,'instrument_id':33,}
                data['altdata'] = {}


                data['altdata']['Reducer'] = 'K-Ryan Hinds'
                data['altdata']['Proposal PI'] = 'Dan Perley'
                data['altdata']['exptime'] = sci_exp_time
                data['altdata']['seeing'] = sci_seeing
                if sci_prop!=None:
                    data['altdata']['Proposal ID']=sci_prop
                    if sci_prop in ['JL24A04','JL24B14']:data['altdata']['Proposal PI'] = 'K-Ryan Hinds'
                    elif sci_prop in ['JL24B15']:data['altdata']['Proposal PI'] = 'Jacob Wise'

                if if_stacked==True:data['altdata']['stacked'],data['altdata']['no_in_stack']= True,no_stacked
                

                start_upl_time = time.time()
                attempts=0
                while True:
                    response = requests.post(url=f"https://fritz.science/api/photometry",headers = {'Authorization': f'token {token}'} ,json=data)

                    if response.status_code==200:
                        break
                    if time.time()-start_upl_time>60 and attempts<=1:
                        print(f' Upload timed out, waiting 30s and trying again')
                        time.sleep(30)
                        attempts+=1
                        start_upl_time = time.time()
                    elif time.time()-start_upl_time>60 and attempts>1:
                        print(f' Upload timed out, giving up')
                        break
                
        return data
  
    t = date.today()
    now = datetime.datetime.now()
    current_time = now.strftime("%H:%M:%S")
    data_mjd = np.round(data['mjd'],6)
    smallest_mjd=np.min(MJD)
    if len(MJD)==1:
        MJD = [data_mjd+0.000001,data_mjd,data_mjd-0.000001]
    else:
        for m in range(len(MJD)):
            MJD.append(MJD[m]+0.000001)
            MJD.append(MJD[m]-0.000001)

    MJD = np.sort(MJD)
    
    try:
        all_fritz_data = SN_data_phot(name)
        # print(all_fritz_data)
        # if len(all_fritz_data)==0:
        #     all_fritz_data = SN_data_phot(name[2:])
        fritz_ind = [ind for ind,val in enumerate(all_fritz_data['instrument_id']) if val==33 and all_fritz_data['filter'].iloc[ind]==data['filter']]

        on_fritz_mjd = [np.round(all_fritz_data['mjd'].iloc[ind],6) for ind in fritz_ind]
        on_fritz_id = [all_fritz_data['id'].iloc[ind] for ind in fritz_ind]
        on_fritz_mag = [np.round(all_fritz_data['mag'].iloc[ind],3) for ind in fritz_ind]
        upload_new=True  #identifier to update photometry in the case of stacking

        if if_stacked==True or upfritz_f==True:
            try:
                del_ind = np.abs(on_fritz_mjd-smallest_mjd).argmin()
                del_id,del_mjd,del_mag = on_fritz_id[del_ind], on_fritz_mjd[del_ind], on_fritz_mag[del_ind]
        
                #if there is photometry on fritz that was uploaded with an mjd thats within 0.005 days of this current measurement, we want to delete and replace with the stacked version
                #this uses the delete photometry function and then uploads this new photometric point 
                if abs(np.round(sci_mjd,6)-del_mjd)<0.005:  
                    deletion_dict = {'phot_id':del_id,'mjd':del_mjd}
                    print(f" Updating photometric point with phot ID: {del_id}")
                    upload_new=True
                    delete_photometry(deletion_dict)

                    #update the photometry downloadeds
                    all_fritz_data = SN_data_phot(name)
                    fritz_ind = [ind for ind,val in enumerate(all_fritz_data['instrument_id']) if val==33 and all_fritz_data['filter'].iloc[ind]==data['filter']]

                    on_fritz_mjd = [np.round(all_fritz_data['mjd'].iloc[ind],6) for ind in fritz_ind]
                    on_fritz_id = [all_fritz_data['id'].iloc[ind] for ind in fritz_ind]
                    on_fritz_mag = [np.round(all_fritz_data['mag'].iloc[ind],3) for ind in fritz_ind]



            except:
                print(f" No photometry to delete for {sci_obj} in {data['filter']} at {data['mjd']}, proceeding with upload")

        if any(x in on_fritz_mjd for x in MJD)==True and upload_new!=True:
            print(f" Data taken for {sci_obj} in {data['filter']} at {data['mjd']} already uploaded, not uploading")
            # print(0,'-------------------')
            upload_new=False


        elif (all(x not in on_fritz_mjd for x in MJD)==True and upload_new!=False)==True or (if_stacked==True and upload_new!=False and np.round(sci_mjd,6) not in on_fritz_mjd)==True:
            print(f" Data taken for {sci_obj} in {data['filter']} at {data['mjd']} not uploaded, proceeding with upload")
            
            #uploading data
            start_upl_time = time.time()
            while True:
                #sleep for 5 seconds and try again if the upload fails
                time.sleep(5)
                response = requests.post(url=f"https://fritz.science/api/photometry",headers = {'Authorization': f'token {token}'} ,json=data)
                if response.status_code==200:
                    break
                if time.time()-start_upl_time>60:
                    print(f' Upload timed out, giving up')
                    break
                

            if response.status_code == 200:
                print(f" Successfully uploaded photometry for {name} in {data['filter']} taken at {data['mjd']}, uploaded at {current_time}")
            else:
                print(f" Failed to upload photometry for {name} in {data['filter']}, status code ={response.status_code} ")
        
            return data

        else:
            print(info_b+f" Data taken for {sci_obj} in {data['filter']} at {data['mjd']} already uploaded, not uploading")

            return data  

    except Exception as e:
        print(f" Unable to find Fritz photometry for {sci_obj} "+e)
        print(f" Fritz error for {sci_obj} in {sci_filt} band, (ie. event not on Fritz or error retrieving photometry from Fritz, check token and Fritz status)")

In [70]:
import glob
all_data = glob.glob('/mnt/data1/users/arikhind/phot_data/GW25dm_phot/AT2025bcw*')
# all_data = glob.glob('/mnt/data1/users/arikhind/phot_data/photometry/AT2025bcw*')
all_data

['/mnt/data1/users/arikhind/phot_data/GW25dm_phot/AT2025bcw_g2025-02-10_74528_stacked_photometry.txt',
 '/mnt/data1/users/arikhind/phot_data/GW25dm_phot/AT2025bcw_r2025-02-10_74895_stacked_photometry.txt',
 '/mnt/data1/users/arikhind/phot_data/GW25dm_phot/AT2025bcw_i2025-02-10_75259_stacked_photometry.txt']

In [71]:
# pd.read_csv(all_data[0])

In [72]:
for f in range(len(all_data)):
  d1 = open(all_data[f], 'r')
  data1 = np.asarray(d1.readlines()[0].split(' ')[:-1])
  fits_info = re.sub('/photometry','/phot_fits_info',re.sub("photometry.txt","phot_info.txt",all_data[f]))
  if os.path.exists(fits_info):pass
  else:fits_info = re.sub("photometry.txt","stacked_phot_info.txt",all_data[f])

  # d2 = open(fits_info, 'r')
  # data2 = np.asarray(d2.readlines()[0].split(','))


  ztfid,filt,mjd,mag,mag_err,lim_mag = data1[0],data1[1],data1[2],data1[3],data1[4],data1[5]
  
  # ztfid,filt,mjd,utcstart,mag,mag_err,lim_mag,ra,dec = data1[0],data1[1],data1[2],data2[1],data1[3],data1[4],data1[5],data2[2],data2[3]
  # exp_t,propid,inst,airmass,see,est_see,no_stacked = data2[6],data2[4],data2[5],data2[7],data2[8],data2[9],data2[10]

  # print(all_data[f])
  # print(ztfid)
  # print(filt)
  # print(mjd)
  # print(mag)
  # print(mag_err)
  # print(lim_mag)
  # print(exp_t)
  # print(propid)
  # print(inst)
  # print(no_stacked)

  # print(ztfid)
  print(str(mjd)+','+str(mag)+','+str(mag_err)+','+str(lim_mag)+',ab,'+str(filt)+',',)


# updata = {'sci_obj':ztfid,'sci_filt':filt,'sci_mjd':float(mjd),'sci_prop':propid,'telescope':inst,'sci_exp_time':exp_t,'sci_seeing':see,'SNR':0.02,'mag_all_err':float(mag_err),'lim_mag':float(lim_mag),'mag':float(mag),'if_stacked':2}
# updata = {'sci_obj':ztfid,'sci_filt':filt,'sci_mjd':float(mjd),'sci_prop':propid,'telescope':inst,'sci_exp_time':exp_t,'sci_seeing':see,'SNR':0.02,'mag_all_err':99,'lim_mag':float(lim_mag),'mag':99,'if_stacked':2}

# upload_phot(updata)


60716.862602,20.975,0.307,21.479,ab,sdssg,
60716.866844,20.802,0.389,21.606,ab,sdssr,
60716.871059,20.873,0.435,21.879,ab,sdssi,
